In [ ]:
import ipyvuetify as v
from IPython.display import display
import pandas as pd
from query_transformer import QueryProcessor

# Initialize Query Processor (equivalent to session state initialization in Streamlit)
query_processor = QueryProcessor()
query_processor.initialize()

# Language Model Options
cypher_ll_options = [
    ("Groq", "llama3-70b-8192"),
    ("Groq", "mixtral-8x7b-32768"),
    ("Groq", "gemma2-9b-it"),
    ("Groq", "llama3-groq-70b-8192-tool-use-preview"),
    ("Ollama", "codegemma:instruct")    
]
formatted_options = [f'{provider}: {model}' for provider, model in cypher_ll_options]

# Create Vuetify components for the UI

# Dropdown to select Cypher Language Model
cypher_llm_dropdown = v.Select(
    label='Select a Language Model', 
    items=formatted_options,
    v_model=None
)

# Dropdown to select QA Model
qa_llm_dropdown = v.Select(
    label='Choose the QA Model',
    items=["llama3-70b-8192", "mixtral-8x7b-32768", "gemma2-9b-it", "llama3-groq-70b-8192-tool-use-preview"],
    v_model=None
)

# Text area for user input query
user_query_input = v.Textarea(
    label='Enter your query in natural language:',
    rows=5,
    v_model=''
)

# Submit button
submit_btn = v.Btn(color='primary', children=['Submit'])

# Text areas for displaying the results
cypher_query_display = v.Textarea(label="Generated Cypher Query", readonly=True, rows=4, v_model="")
final_answer_display = v.Textarea(label="Final Answer", readonly=True, rows=4, v_model="")
query_result_table = v.DataTable(headers=[{'text': 'Query Result', 'value': 'result'}], items=[])

# Define function to handle the submission logic
def on_submit_click(widget, event, data):
    user_query = user_query_input.v_model
    if user_query:
        # Get selected provider and LLM from the dropdowns
        provider, llm = cypher_llm_dropdown.v_model.split(': ')
        query_processor.set_cypher_llm_choice(provider, llm)
        query_processor.set_qa_llm_choice(qa_llm_dropdown.v_model)

        # Process the query using QueryProcessor
        cypher_query, cypher_result, final_answer = query_processor.handle_query(user_query)
        
        # Display the results
        cypher_query_display.v_model = cypher_query
        query_result_table.items = [{'result': str(result)} for result in cypher_result]
        final_answer_display.v_model = final_answer
    else:
        cypher_query_display.v_model = "Please enter a query."

# Attach event handler to the button click event
submit_btn.on_event('click', on_submit_click)

# Layout the components using Vuetify's grid system
main_content = v.Container(
    children=[
        v.Row(
            children=[
                v.Col(cols=4, children=[
                    cypher_llm_dropdown,
                    qa_llm_dropdown
                ]),
                v.Col(cols=8, children=[
                    user_query_input,
                    submit_btn,
                    cypher_query_display,
                    query_result_table,
                    final_answer_display
                ])
            ]
        )
    ]
)

# Display the layout in the notebook
v_app = v.App(children=[main_content])
display(v_app)
